In [30]:
import random
from typing import List, Union

import numpy as np
from Crypto.Util.number import getRandomNBitInteger
from numpy.polynomial.polynomial import polyadd, polymul

In [31]:
def polydiv(poly1, poly2):
    """
    Divide poly1 by poly2 and return the quotient and remainder as numpy arrays
    GENERATED BY CHATGPT
    """
    # Ensure that the degree of the second polynomial is less than or equal to the degree of the first polynomial
    while len(poly1) >= len(poly2):
        # Compute the quotient of the leading terms of the two polynomials
        quotient = poly1[-1] // poly2[-1]

        # Compute the product of the quotient and the second polynomial shifted to align with the leading term of the first polynomial
        product = np.zeros(len(poly1), dtype=object)
        product[-len(poly2) :] = quotient * poly2

        # Compute the difference between the first polynomial and the product
        difference = poly1 - product

        # Remove any leading zeros from the difference
        while len(difference) > 0 and difference[-1] == 0:
            difference = difference[:-1]

        # Set the difference as the new value of the first polynomial
        poly1 = difference

    # Return the quotient and remainder as numpy arrays
    quotient = np.zeros(len(poly1) + len(poly2) - 1, dtype=object)
    quotient[-len(poly1) :] = poly1
    remainder = poly1
    return quotient, remainder



In [32]:
def roundv(array):
    return np.array([round(a) for a in array], dtype=object)


def intv(array):
    return np.array([int(a) for a in array], dtype=object)

In [33]:
def int2base(x: int, base: int) -> List[int]:
    digits = []
    while x > 0:
        q, r = divmod(x, base)
        digits.append(r)
        x = q
    return digits

In [34]:
def mod_center(x, m: int, left_closed: bool = True):
    # [-m // 2, m // 2)
    if left_closed:
        return (x + m // 2) % m - m // 2
    # (-m // 2, m // 2]
    else:
        return (x + m // 2 - 1) % m - m // 2 + 1

## QuotientRingPoly

In [35]:
def init_poly_modulus(poly_modulus: Union[int, np.array]) -> np.array:
    # If it's int, let it be x ^ poly_modulus + 1, else just init the poly modulus
    if isinstance(poly_modulus, int):
        poly_modulus = np.array([1] + [0] * (poly_modulus - 1) + [1], dtype=object)
    else:
        poly_modulus = poly_modulus
    return poly_modulus


def untrim_seq(poly: np.array, degree: int) -> np.array:
    # Add 0s to the higher powers until we reach degree
    coef = np.append(poly, [0] * (degree - len(poly)))
    return coef

In [36]:
class QuotientRingPoly:
    def __init__(
        self,
        coef: np.array,
        coef_modulus: int,
        poly_modulus: Union[int, np.array],
    ):
        self._coef_modulus = coef_modulus
        self._poly_modulus = init_poly_modulus(poly_modulus)
        self._coef = coef
        # Reduce mod coef and mod poly.
        self.degree = len(poly_modulus) - 1
        self._reduce()

    def _reduce(self):
        self._coef = roundv(self._coef)
        self._coef = mod_center(self._coef, self.coef_modulus)
        _, self._coef = polydiv(self._coef, self.poly_modulus)
        self._coef = mod_center(self._coef, self.coef_modulus)

        # Extend the 0s to match the degree
        self._coef = untrim_seq(self._coef, self.degree)
        # Transform the coeffs to int, sanity check.
        self._coef = roundv(self._coef)

    def _check_qring(self, other):
        if (
            any(self.poly_modulus != other.poly_modulus)
            or self.coef_modulus != other.coef_modulus
        ):
            raise ValueError("Polynomials are not in the same Quotient Ring")

    def __neg__(self):
        return QuotientRingPoly(-self._coef, self.coef_modulus, self.poly_modulus)

    def __add__(self, other):
        # Perform addition. If other is int, add to coeff
        if isinstance(other, (int, float)):
            res_coef = self.coef + other
            res = QuotientRingPoly(res_coef, self.coef_modulus, self.poly_modulus)
        else:
            self._check_qring(other)
            res_poly = polyadd(self._coef, other.coef)
            res = QuotientRingPoly(res_poly, self.coef_modulus, self.poly_modulus)
        return res

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self, other):
        if isinstance(other, (int, float)):
            res_coef = self.coef * other
            res = QuotientRingPoly(res_coef, self.coef_modulus, self.poly_modulus)
        else:
            self._check_qring(other)
            # print(np.polymul(self.poly.coef, other.poly.coef))
            res_coef = polymul(self._coef, other.coef)
            res = QuotientRingPoly(res_coef, self.coef_modulus, self.poly_modulus)
        return res

    def __floordiv__(self, other):
        if isinstance(other, (int, float)):
            res_coef = self.coef // other
            res = QuotientRingPoly(res_coef, self.coef_modulus, self.poly_modulus)
        else:
            self._check_qring(other)
            q, _ = polydiv(self.poly, other.poly)
            res = QuotientRingPoly(q, self.coef_modulus, self.poly_modulus)
        return res

    def __mod__(self, other):
        if isinstance(other, (int, float)):
            res_coef = self.coef % other
            res = QuotientRingPoly(res_coef, self.coef_modulus, self.poly_modulus)
        else:
            self._check_qring(other)
            _, r = polydiv(self.poly, other.poly)
            res = QuotientRingPoly(r, self.coef_modulus, self.poly_modulus)
        return res

    def __eq__(self, other):
        return (
            all(self.poly_modulus == other.poly_modulus)
            and self.coef_modulus == other.coef_modulus
            and all(self._coef == other.coef)
        )

    def copy(self) -> "QuotientRingPoly":
        return QuotientRingPoly(self.coef, self.coef_modulus, self.poly_modulus)

    @property
    def poly_modulus(self):
        return self._poly_modulus.copy()

    @property
    def coef_modulus(self):
        return self._coef_modulus

    @coef_modulus.setter
    def coef_modulus(self, value):
        self._coef_modulus = value
        self._reduce()

    @property
    def coef(self):
        return self._coef

    @coef.setter
    def coef(self, value):
        self._coef = value
        self._reduce()

    def __repr__(self):
        r = f"{self.coef}, {self.coef_modulus}, {self.poly_modulus}"
        return r

In [37]:
def random_binary_poly(
    coef_modulus: int, poly_modulus: Union[int, np.array]
) -> QuotientRingPoly:
    """Generate a random binary polynomial in the given quotient ring."""
    poly_modulus = init_poly_modulus(poly_modulus)
    size = len(poly_modulus) - 1
    coef = np.random.choice([0, 1], size=size, dtype=object)
    return QuotientRingPoly(coef, coef_modulus, poly_modulus)


def random_ternary_poly(
    coef_modulus: int, poly_modulus: Union[int, np.array]
) -> QuotientRingPoly:
    """Generate a random ternary polynomial in the given quotient ring."""
    poly_modulus = init_poly_modulus(poly_modulus)
    size = len(poly_modulus) - 1

    # 0 with 1/2 chance, -1 or 1 with 1/2 chance
    # coef = np.random.randint(-1, 2, size, dtype=int)
    coef = np.random.choice(
        np.array([-1, 0, 1], dtype=object), size=size, p=[1 / 4, 1 / 2, 1 / 4]
    )
    return QuotientRingPoly(coef, coef_modulus, poly_modulus)


def random_uniform_poly(
    coef_modulus: int,
    poly_modulus: Union[int, np.array],
    high=None,
) -> QuotientRingPoly:
    """Generate a random polynomial with discrete coefficients uniformly distributed in the given quotient ring."""

    if high is None:
        high = coef_modulus - 1
    poly_modulus = init_poly_modulus(poly_modulus)
    size = len(poly_modulus) - 1
    coef = np.array([random.randrange(0, high) for _ in range(size)], dtype=object)
    # coef = np.random.randint(0, coef_modulus, size, dtype=object)
    return QuotientRingPoly(coef, coef_modulus, poly_modulus)


def random_normal_poly(
    coef_modulus: int,
    poly_modulus: Union[int, np.array],
    mu: float = 0,
    std: float = 3.8,
) -> QuotientRingPoly:
    """Generate a random polynomial with discrete coefficients extracted from a normal distribution in the given quotient ring."""
    poly_modulus = init_poly_modulus(poly_modulus)
    size = len(poly_modulus) - 1
    coef = np.array([round(random.gauss(mu, std)) for _ in range(size)], dtype=object)
    return QuotientRingPoly(coef, coef_modulus, poly_modulus)

## Encryption Scheme

In [38]:
def gen_secret_key(coef_modulus: int, poly_modulus: np.ndarray):
    # Draw the secret
    s = random_ternary_poly(coef_modulus, poly_modulus)
    return s


def gen_public_key(
    sk: QuotientRingPoly,
    coef_modulus: int,
    poly_modulus: np.ndarray,
    plaintext_modulus: int,
):
    # Generate noise e
    e = random_normal_poly(coef_modulus, poly_modulus)
    # Generate unifrom poly a
    a = random_uniform_poly(coef_modulus, poly_modulus)
    # RLWE instance b
    b = a * sk + e * plaintext_modulus
    return b, -a


def encrypt(
    msg: QuotientRingPoly,
    pk0: QuotientRingPoly,
    pk1: QuotientRingPoly,
    coef_modulus: int,
    poly_modulus: np.ndarray,
    plaintext_modulus: int,
):
    u = random_ternary_poly(coef_modulus, poly_modulus)
    e0 = random_normal_poly(coef_modulus, poly_modulus)
    e1 = random_normal_poly(coef_modulus, poly_modulus)
    # Mask the message with a rlwe instance (b * r + te)
    c0 = pk0 * u + e0 * plaintext_modulus + msg
    c1 = pk1 * u + e1 * plaintext_modulus
    return c0, c1


def decrypt(
    c0: QuotientRingPoly,
    c1: QuotientRingPoly,
    sk: QuotientRingPoly,
    plaintext_modulus: int,
    return_noise: bool = False,
):
    msg = c0 + c1 * sk
    noise = np.max(np.abs(msg.coef))
    msg = msg % plaintext_modulus

    if return_noise:
        msg_not_reduced = np.max((c0 + c1 * sk).coef)
        # t = mod_center(polymul(c0.coef, sk.coef), c0.coef_modulus)
        # t = mod_center(polyadd(c1.coef, t), c0.coef_modulus)
        # _, t = polydiv(t, c0.poly_modulus)
        # msg_not_reduced = mod_center(t, c0.coef_modulus)
        noise = np.max(np.abs(msg_not_reduced))

        return msg, noise
    else:
        return msg

In [39]:
n = 2**4
# q
coef_modulus = getRandomNBitInteger(32)
# coef_modulus = 874
# coef_modulus = getrandbits(12)
plaintext_modulus = 7

poly_modulus = init_poly_modulus(n)
print(coef_modulus)
print(poly_modulus)


3969935913
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [40]:
# Generate secret key
sk = gen_secret_key(coef_modulus, poly_modulus)
sk

[1 0 1 0 0 1 -1 1 1 0 1 -1 0 1 1 0], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

In [41]:
# Generate public key pair
pk0, pk1 = gen_public_key(sk, coef_modulus, poly_modulus, plaintext_modulus)
pk0, pk1

([1217923045 -237228940 1535034177 -124370598 -1959090897 1572265328
  -371993780 1982876502 -1685201548 1747962457 1077578187 1488253177
  357926972 -1295677644 76141725 -688045230], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],
 [1106350395 1864040189 1584176335 153112283 1469710713 -1821698576
  1069553181 297952895 137848659 77872641 1292123888 -561575081 -1034435618
  -997267311 -666289259 -121547467], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1])

## FHE

In [42]:
def add(c0, c1, c02, c12):
    return c0 + c02, c1+ c12

## Original data

In [43]:
m2 = 1

# Convert the integer to its coefficients in base plaintext_modulus
msg_coef2 = int2base(m2, plaintext_modulus)

# Pad the coefficients with zeros to match the polynomial degree
msg_coef2 = np.pad(msg_coef2, (0, n - len(msg_coef2)), mode='constant')

# Create the QuotientRingPoly object
msg2 = QuotientRingPoly(msg_coef2, coef_modulus, poly_modulus)
print(msg2)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [44]:
# Encrypt the message into two ciphertexts
c02, c12 = encrypt(msg2, pk0, pk1, coef_modulus, poly_modulus, plaintext_modulus)
c02, c12

([1030125794 -785467988 -46913582 1306443531 613423492 -29675952
  -1761112419 1221805753 -1333787029 -90398085 -631977931 -1014015054
  -1189074021 -1256950152 -1125322365 -225782537], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],
 [1915437772 1753194000 151145338 -568822212 38259791 -501917917 -391473088
  1652723620 1714635418 1558782861 -706256133 -545262021 -104236873
  -401238734 -181616893 -1839460116], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1])

In [45]:
import numpy as np
import pywt
import hashlib
import imageio.v2 as imageio
from scipy.stats import entropy

# Read the image file
image = imageio.imread('d1.jpg')

# Convert the image to grayscale if it's not already
if image.ndim == 3:
    image = np.mean(image, axis=2)

# Compute histogram values
hist_values, _ = np.histogram(image, bins='auto')

# Compute wavelet coefficients using different wavelet functions
coeffs_haar = pywt.wavedec2(image, 'haar')
coeffs_db = pywt.wavedec2(image, 'db1') # Daubechies 1 (db1) is equivalent to Haar
coeffs_sym = pywt.wavedec2(image, 'sym2') # Symlets 2
coeffs_bior = pywt.wavedec2(image, 'bior1.3') # Biorthogonal 1.3

# Flatten the coefficients for concatenation
coeffs_haar_flat = np.concatenate([arr.ravel() for arr in coeffs_haar[0]])
coeffs_db_flat = np.concatenate([arr.ravel() for arr in coeffs_db[0]])
coeffs_sym_flat = np.concatenate([arr.ravel() for arr in coeffs_sym[0]])
coeffs_bior_flat = np.concatenate([arr.ravel() for arr in coeffs_bior[0]])

# Compute Mean, Median, Entropy, Standard Deviation
mean_val = np.mean(image)
median_val = np.median(image)
entropy_val = entropy(hist_values)
std_dev_val = np.std(image)

# Concatenate all values into a single string
concatenated_values = ''.join(map(str, [
    hist_values, coeffs_haar_flat, coeffs_db_flat, coeffs_sym_flat, 
    coeffs_bior_flat, mean_val, median_val, entropy_val, std_dev_val
]))

# Hash the concatenated string to get a new integer
integer_value = int(hashlib.sha256(concatenated_values.encode()).hexdigest(), 16)

print(f"The hashed integer value is: {integer_value}")

The hashed integer value is: 68215188356612381213524994025934183644187897378700032362900265715051519959448


In [46]:

# Convert integer value to string
integer_str = str(integer_value)

# Define the number of digits per element
digits_per_element = 13

# Calculate the number of elements needed
num_elements = len(integer_str) // digits_per_element + (1 if len(integer_str) % digits_per_element != 0 else 0)

# Create a list to store the divided elements
new_list = []

# Divide the integer value into elements of specified length
for i in range(num_elements):
    start_index = i * digits_per_element
    end_index = start_index + digits_per_element
    new_list.append(int(integer_str[start_index:end_index]))

print("Divided elements:", new_list)

Divided elements: [6821518835661, 2381213524994, 259341836441, 8789737870003, 2362900265715, 51519959448]


In [47]:
# Initialize an empty list
decrypted_messages = []

for m in new_list:
    # Convert the integer to its coefficients in base plaintext_modulus
    msg_coef = int2base(m, plaintext_modulus)
    
    # Pad the coefficients with zeros to match the polynomial degree
    msg_coef = np.pad(msg_coef, (0, n - len(msg_coef)), mode='constant')
    
    # Create the QuotientRingPoly object
    msg = QuotientRingPoly(msg_coef, coef_modulus, poly_modulus)
    
    # Perform further operations with 'msg' if needed
    # For example, you can print the coefficients:
    # print("Coefficients of the polynomial:", msg.coef)
    c0, c1 = encrypt(msg, pk0, pk1, coef_modulus, poly_modulus, plaintext_modulus)

    c0_res, c1_res = add(c0, c1, c02, c12)
    msg_res_decr, noise = decrypt(c0_res, c1_res, sk, plaintext_modulus, return_noise=True)
    
    # Append the decrypted message to the list
    decrypted_messages.append(msg_res_decr.coef.astype(int))
    noise < coef_modulus // 2

# Now 'decrypted_messages' contains the decrypted coefficients of all messages
print(decrypted_messages)


[array([6, 3, 3, 4, 5, 2, 1, 1, 4, 0, 6, 5, 2, 0, 3, 1]), array([4, 4, 6, 0, 0, 0, 6, 5, 4, 5, 1, 0, 4, 3, 3, 0]), array([3, 4, 3, 4, 4, 1, 6, 0, 5, 0, 1, 5, 4, 2, 0, 0]), array([0, 1, 0, 4, 0, 1, 0, 2, 6, 5, 1, 0, 5, 6, 5, 1]), array([4, 2, 3, 3, 0, 6, 4, 0, 6, 6, 6, 4, 2, 3, 3, 0]), array([0, 5, 2, 0, 0, 3, 6, 6, 4, 2, 0, 5, 3, 0, 0, 0])]


In [48]:
noise < coef_modulus // 2


True

## input

In [49]:
n = 2**4
# q
coef_modulus_ = getRandomNBitInteger(32)
# coef_modulus = 874
# coef_modulus = getrandbits(12)
plaintext_modulus_ = 7

poly_modulus_ = init_poly_modulus(n)
print(coef_modulus_)
print(poly_modulus_)

2815834484
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [50]:
# Generate secret key
sk_ = gen_secret_key(coef_modulus_, poly_modulus_)
sk_

[0 1 0 0 0 -1 1 0 0 -1 0 0 -1 0 1 -1], 2815834484, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

In [51]:
# Generate public key pair
pk0_, pk1_ = gen_public_key(sk_, coef_modulus_, poly_modulus_, plaintext_modulus_)
pk0_, pk1_

([474798674 -1164528790 -250916070 889128195 13148117 431277323 604657965
  823805469 -166969269 933742934 617985778 -75886796 404816893 -1233072685
  580497739 613503566], 2815834484, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],
 [-799828719 -73584817 -1241497455 92350455 -1048754483 -854426665
  1179410051 623637602 -247932636 555100180 1155474498 -84648144
  -1315752371 -1051032127 532387314 -22528218], 2815834484, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1])

In [52]:
m2_ = 1

# Convert the integer to its coefficients in base plaintext_modulus
msg_coef2_ = int2base(m2_, plaintext_modulus)

# Pad the coefficients with zeros to match the polynomial degree
msg_coef2_ = np.pad(msg_coef2_, (0, n - len(msg_coef2_)), mode='constant')

# Create the QuotientRingPoly object
msg2_ = QuotientRingPoly(msg_coef2_, coef_modulus, poly_modulus)

In [53]:
# Encrypt the message into two ciphertexts
c02_, c12_ = encrypt(msg2_, pk0, pk1, coef_modulus, poly_modulus, plaintext_modulus)
c02_, c12_

([851632861 -1671503504 92760275 1146404368 373197493 -1316773397
  -384764042 -532112619 653699363 845958833 872212423 -541969272 546110815
  -970728569 1277673745 -1304185417], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],
 [-553706901 -1400814939 493415988 100681910 -564920784 -707744296
  -766111760 507573792 262704368 -222396787 498838631 -1334268269 531095418
  -1053407054 1554994564 1916301503], 3969935913, [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1])

In [54]:
import numpy as np
import pywt
import hashlib
import imageio.v2 as imageio
from scipy.stats import entropy

# Read the image file
image_ = imageio.imread('d2.jpg')

# Convert the image to grayscale if it's not already
if image_.ndim == 3:
    image_ = np.mean(image_, axis=2)

# Compute histogram values
hist_values_, _ = np.histogram(image_, bins='auto')

# Compute wavelet coefficients using different wavelet functions
coeffs_haar_ = pywt.wavedec2(image_, 'haar')
coeffs_db_ = pywt.wavedec2(image_, 'db1') # Daubechies 1 (db1) is equivalent to Haar
coeffs_sym_ = pywt.wavedec2(image_, 'sym2') # Symlets 2
coeffs_bior_ = pywt.wavedec2(image_, 'bior1.3') # Biorthogonal 1.3

# Flatten the coefficients for concatenation
coeffs_haar_flat_ = np.concatenate([arr.ravel() for arr in coeffs_haar_[0]])
coeffs_db_flat_ = np.concatenate([arr.ravel() for arr in coeffs_db_[0]])
coeffs_sym_flat_ = np.concatenate([arr.ravel() for arr in coeffs_sym_[0]])
coeffs_bior_flat_ = np.concatenate([arr.ravel() for arr in coeffs_bior_[0]])

# Compute Mean, Median, Entropy, Standard Deviation
mean_val_ = np.mean(image_)
median_val_ = np.median(image_)
entropy_val_ = entropy(hist_values_)
std_dev_val_ = np.std(image_)

# Concatenate all values into a single string
concatenated_values_ = ''.join(map(str, [
    hist_values_, coeffs_haar_flat_, coeffs_db_flat_, coeffs_sym_flat_, 
    coeffs_bior_flat_, mean_val, median_val_, entropy_val_, std_dev_val_
]))


# Hash the concatenated string to get a new integer
integer_value_ = int(hashlib.sha256(concatenated_values_.encode()).hexdigest(), 16)

print(f"The hashed integer value is: {integer_value_}")

The hashed integer value is: 88689577949153426393034791955192935107782469724695559826869271080524953543102


In [55]:
# Convert integer value to string
integer_str_ = str(integer_value_)

# Define the number of digits per element
digits_per_element_ = 13

# Calculate the number of elements needed
num_elements_ = len(integer_str_) // digits_per_element_ + (1 if len(integer_str_) % digits_per_element_ != 0 else 0)

# Create a list to store the divided elements
new_list_ = []

# Divide the integer value into elements of specified length
for i in range(num_elements_):
    start_index_ = i * digits_per_element_
    end_index_ = start_index_ + digits_per_element_
    new_list_.append(int(integer_str_[start_index_:end_index_]))

print("Divided elements:", new_list_)

Divided elements: [8868957794915, 3426393034791, 9551929351077, 8246972469555, 9826869271080, 524953543102]


In [56]:
# Initialize an empty list
decrypted_messages_ = []

for m_ in new_list_:
    # Convert the integer to its coefficients in base plaintext_modulus
    msg_coef_ = int2base(m_, plaintext_modulus)
    
    # Pad the coefficients with zeros to match the polynomial degree
    msg_coef_ = np.pad(msg_coef_, (0, n - len(msg_coef_)), mode='constant')
    
    # Create the QuotientRingPoly object
    msg_ = QuotientRingPoly(msg_coef_, coef_modulus, poly_modulus)
    
    # Perform further operations with 'msg' if needed
    # For example, you can print the coefficients:
    # print("Coefficients of the polynomial:", msg.coef)
    c0_, c1_ = encrypt(msg_, pk0, pk1, coef_modulus, poly_modulus, plaintext_modulus)

    c0_res_, c1_res_ = add(c0_, c1_, c02_, c12_)
    msg_res_decr_, noise = decrypt(c0_res_, c1_res_, sk, plaintext_modulus, return_noise=True)
    
    # Append the decrypted message to the list
    decrypted_messages_.append(msg_res_decr_.coef.astype(int))
    noise < coef_modulus // 2

# Now 'decrypted_messages' contains the decrypted coefficients of all messages
print(decrypted_messages_)

[array([2, 5, 1, 6, 5, 2, 0, 2, 0, 2, 2, 5, 3, 0, 6, 1]), array([0, 0, 3, 2, 5, 1, 3, 3, 1, 6, 5, 3, 2, 0, 5, 0]), array([2, 3, 6, 3, 6, 5, 6, 6, 4, 0, 5, 0, 4, 0, 0, 2]), array([2, 1, 5, 1, 4, 2, 4, 4, 4, 2, 5, 5, 0, 1, 5, 1]), array([2, 0, 5, 5, 1, 4, 0, 6, 6, 2, 5, 6, 2, 3, 0, 2]), array([3, 2, 0, 1, 3, 3, 0, 6, 5, 2, 3, 6, 2, 5, 0, 0])]


## Comparision

In [57]:
are_lists_equal = all(np.array_equal(a, b) for a, b in zip(decrypted_messages, decrypted_messages_))

# Print the result
print("Are both lists of arrays equal?", are_lists_equal)

Are both lists of arrays equal? False


## parameters

In [58]:
import time
import psutil

def get_cpu_memory_usage():
    # Get CPU utilization
    cpu_percent = psutil.cpu_percent(interval=1)

    # Get memory utilization
    memory = psutil.virtual_memory()
    memory_percent = memory.percent

    return cpu_percent, memory_percent

def main():
    start_time = time.time()

    # Your main code goes here
    # ...

    # Simulate some processing time
    time.sleep(2)  # Sleep for 2 seconds

    # Get execution time
    execution_time = time.time() - start_time

    # Get CPU and memory usage
    cpu_percent, memory_percent = get_cpu_memory_usage()

    # Print results
    print("Execution time:", execution_time, "seconds")
    print("CPU utilization:", cpu_percent, "%")
    print("Memory utilization:", memory_percent, "%")

if __name__ == "__main__":
    main()


Execution time: 2.004199743270874 seconds
CPU utilization: 0.9 %
Memory utilization: 84.9 %
